# Practica Web Scraping

Leonardo Quishpe

hacer un sistema de recuperacion de informacion con 100 recetashaciendo web scraping

Web scrapping allrecipes.com

In [1]:
from bs4 import BeautifulSoup


In [2]:
file = 'data/receta.html'

with open(file, 'r', encoding='utf-8') as f:
    html_content = f.read()
    
soup = BeautifulSoup(html_content, 'html.parser')

Descargar otras paginas

In [3]:
recipes_links = soup.find_all("a")
recipe_urls = []
for link in recipes_links:
    href = link["href"]
    if "recipe" in href:
        recipe_urls.append(href)
        
print("Linked recipes:")
for url in recipe_urls:
    print(url)
print(len(recipe_urls))


Linked recipes:
https://www.allrecipes.com/authentication/login?regSource=3675&relativeRedirectUrl=%2Frecipe%2F20171%2Fquick-and-easy-pizza-crust%2F
/account/add-recipe
https://www.myrecipes.com/favorites
https://www.allrecipes.com/authentication/logout?relativeRedirectUrl=%2Frecipe%2F20171%2Fquick-and-easy-pizza-crust%2F
https://www.magazines.com/allrecipes-magazine.html?utm_source=allrecipes.com&utm_medium=owned&utm_campaign=i111arr1w2661
https://www.magazines.com/allrecipes-magazine.html
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/
https://www.allrecipes.com/re

Bola de Nieve (Scraping)

In [4]:
title = soup.find("meta", {"property": "og:title"})["content"]
print(title)

Easy Homemade Pizza Dough


In [5]:
ingredients_section = soup.find_all("li", class_='mm-recipes-structured-ingredients__list-item')
for ingredient in ingredients_section:
    print(ingredient.text.strip())

1 cup warm water (110 degrees F/45 degrees C)
1 (.25 ounce) package active dry yeast
1 teaspoon white sugar
2 ½ cups bread flour
2 tablespoons olive oil
1 teaspoon salt


In [6]:
rating = soup.find("div", class_="comp mm-recipes-review-bar__rating mntl-text-block text-label-300").text.strip()
print(rating)

4.7


In [7]:
# Busca todos los bloques de tiempo
items = soup.find_all("div", class_="mm-recipes-details__item")

# Recorre cada uno buscando el que tenga la etiqueta "Total Time:"
for item in items:
    label = item.find("div", class_="mm-recipes-details__label")
    value = item.find("div", class_="mm-recipes-details__value")   
    if label and "Total Time:" in label.get_text(strip=True):
        total_time = value.get_text(strip=True)
        print("Total Time:", total_time)
    if label and "Servings" in label.get_text(strip=True):
        servings = value.get_text(strip=True)
        print("Servings:", servings)
        break
        


Total Time: 45 mins
Servings: 8


In [8]:
#summary
# Buscar todos los bloques de texto
paragraphs = soup.find_all("p")

for p in paragraphs:
    text = p.get_text(strip=True)
    if len(text) > 100:  
        summary = text
        break

print(summary)


This is a great recipe when you don't want to wait for the dough to rise. You just mix it and allow it to rest for 5 minutes and then it's ready to go! It yields a soft, chewy crust.


In [9]:
# Extraer pasos de preparación sin créditos
preparation_steps = []

step_items = soup.select("li.mntl-sc-block-group--LI")

for i, item in enumerate(step_items, 1):
    text = item.get_text(" ", strip=True)
    if text and len(text) > 20:
        # Eliminar texto no deseado si está al final
        if text.endswith("Dotdash Meredith Food Studios"):
            text = text.replace("Dotdash Meredith Food Studios", "").strip()
        preparation_steps.append(f"step {i}: {text}")

# Imprimir los pasos limpios
for step in preparation_steps:
    print(step)

step 1: Gather all ingredients. Preheat oven to 450 degrees F (230 degrees C), and lightly grease a pizza pan.
step 2: Place warm water in a bowl; add yeast and sugar. Mix and let stand until creamy, about 10 minutes.
step 3: Add flour, oil, and salt to the yeast mixture; beat until smooth. You can do this by hand or use a stand mixer fitted with a dough hook to make it easier.
step 4: Let rest for 5 minutes.
step 5: Turn dough out onto a lightly floured surface and pat or roll into a 12-inch circle.
step 6: Transfer to the prepared pizza pan.
step 7: Spread crust with sauce and toppings of your choice.
step 8: Bake in the preheated oven until golden brown, 15 to 20 minutes. Remove from the oven and let cool for 5 minutes before serving. Allrecipes / Abbey Littlejohn


In [10]:
# Buscar las filas de la tabla de nutrición
rows = soup.select("tr.mm-recipes-nutrition-facts-summary__table-row")

nutrition_facts = {}
for row in rows:
    columns = row.find_all("td")
    if len(columns) >= 2:
        label = columns[1].get_text(strip=True)
        value = columns[0].get_text(strip=True)
        nutrition_facts[label] = value

# Imprimir resultados
for k, v in nutrition_facts.items():
    print(f"{k}: {v}")

Calories: 170
Fat: 4g
Carbs: 28g
Protein: 5g


In [12]:
%pip install ace_tools_open

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 31.7 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [itables]
   ---------------------------------------- 2/2 [ace_tools_open]

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import ace_tools_open as tools
# Create DataFrame
df = pd.DataFrame([{
    "title": title,
    "summary": summary,
    "ingredients": ingredients_section,
    "steps": preparation_steps,
    "total_time": total_time,
    "servings": servings,
    "rating": rating,
    "nutrition": nutrition_facts
}])


tools.display_dataframe_to_user(name="Recetas", dataframe=df)

Recetas


Loading ITables v2.4.4 from the internet... (need help?)


Para 100 links

In [14]:
import requests
import time
from bs4 import BeautifulSoup

recipe_data = []

headers = {"User-Agent": "Mozilla/5.0"}

for url in recipe_urls:
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        title = soup.find("meta", {"property": "og:title"})
        title = title["content"] if title else "Título no encontrado"

        summary = next(
            (p.get_text(strip=True) for p in soup.find_all("p") if len(p.get_text(strip=True)) > 100),
            "Resumen no encontrado"
        )

        ingredients = " | ".join([
            ing.text.strip()
            for ing in soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
        ]) or "Ingredientes no encontrados"

        steps = " ".join([
            f"step {i+1}: {item.get_text(' ', strip=True)}"
            for i, item in enumerate(soup.select("li.mntl-sc-block-group--LI"))
        ]) or "Pasos no encontrados"

        total_time = next(
            (value.text.strip() for label, value in zip(
                soup.find_all("div", class_="mm-recipes-details__label"),
                soup.find_all("div", class_="mm-recipes-details__value"))
             if "Total Time" in label.text),
            "Tiempo no encontrado"
        )

        servings = next(
            (value.text.strip() for label, value in zip(
                soup.find_all("div", class_="mm-recipes-details__label"),
                soup.find_all("div", class_="mm-recipes-details__value"))
             if "Servings" in label.text),
            "Porciones no encontradas"
        )

        rating_tag = soup.find("div", class_="comp mm-recipes-review-bar__rating mntl-text-block text-label-300")
        rating = rating_tag.text.strip() if rating_tag else "Rating no encontrado"

        nutrition = " | ".join([
            f"{row.find_all('td')[1].get_text(strip=True)}: {row.find_all('td')[0].get_text(strip=True)}"
            for row in soup.select("tr.mm-recipes-nutrition-facts-summary__table-row")
            if len(row.find_all('td')) >= 2
        ]) or "Nutrición no encontrada"

        # Agregar a la lista
        recipe_data.append({
            "url": url,
            "title": title,
            "summary": summary,
            "ingredients": ingredients,
            "steps": steps,
            "total_time": total_time,
            "servings": servings,
            "rating": rating,
            "nutrition": nutrition
        })

        print(f"✅ Receta extraída: {title}")

    except Exception as e:
        print(f"❌ Error en {url}: {e}")

    time.sleep(1)  # evitar sobrecarga

# Mostrar un ejemplo
print("\nEjemplo de receta extraída:")
print(recipe_data[0])


✅ Receta extraída: Título no encontrado
❌ Error en /account/add-recipe: Invalid URL '/account/add-recipe': No scheme supplied. Perhaps you meant https:///account/add-recipe?
✅ Receta extraída: Título no encontrado
✅ Receta extraída: Easy Homemade Pizza Dough
✅ Receta extraída: Allrecipes Magazine Subscription | Subscribe to Allrecipes Magazine
✅ Receta extraída: Allrecipes Magazine Subscription | Subscribe to Allrecipes Magazine
✅ Receta extraída: Dinner Recipes
✅ Receta extraída: 5 Ingredient Main Dishes
✅ Receta extraída: One-Pot Meal Recipes
✅ Receta extraída: Quick and Easy Recipes
✅ Receta extraída: 30-Minute Meals
✅ Receta extraída: Family Dinner Recipes
✅ Receta extraída: Soups, Stews and Chili Recipes
✅ Receta extraída: Comfort Food
✅ Receta extraída: Main Dishes
✅ Receta extraída: Sheet Pan Dinner Recipes
✅ Receta extraída: Dinner Recipes
✅ Receta extraída: Recipes A-Z
✅ Receta extraída: Breakfast and Brunch Recipes
✅ Receta extraída: Lunch Recipes
✅ Receta extraída: Healthy R

In [15]:
df = pd.DataFrame(recipe_data)
tools.display_dataframe_to_user(name="Recetas", dataframe=df)

Recetas


Loading ITables v2.4.4 from the internet... (need help?)
